# Wine dataset example model
### https://predictivemodeler.com/2019/10/19/tensorflow-boston-house-prices/

In [62]:
from sklearn.datasets import load_boston
import pandas as pd
boston = load_boston()

In [63]:
data = pd.DataFrame(boston.data, columns=boston.feature_names)
data["MEDV"] = boston.target

In [64]:
train_dataset = data.sample(frac=0.8,random_state=0)
test_dataset = data.drop(train_dataset.index)
train_dataset.keys()

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [65]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
  model = keras.Sequential([
    layers.Dense(50, activation='relu', input_shape=[len(train_dataset.keys())-1]),
    layers.Dense(50, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [66]:
model = build_model();

In [67]:
train_stats = train_dataset.describe()
train_stats.pop("MEDV")
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
CRIM,405.0,3.670339,8.977954,0.00632,0.08265,0.26838,3.47428,88.9762
ZN,405.0,11.067901,22.688396,0.00000,0.00000,0.00000,12.50000,95.0000
INDUS,405.0,11.097951,6.699157,0.74000,5.32000,9.69000,18.10000,27.7400
CHAS,405.0,0.069136,0.253999,0.00000,0.00000,0.00000,0.00000,1.0000
NOX,405.0,0.553098,0.115564,0.38500,0.44900,0.53800,0.62400,0.8710
RM,405.0,6.269768,0.690552,3.56100,5.87800,6.19300,6.61900,8.7800
AGE,405.0,68.241975,27.908527,2.90000,45.00000,76.50000,94.10000,100.0000
DIS,405.0,3.834098,2.162173,1.12960,2.07880,3.27210,5.21460,12.1265
RAD,405.0,9.474074,8.618564,1.00000,4.00000,5.00000,24.00000,24.0000
TAX,405.0,405.839506,164.057951,187.00000,284.00000,334.00000,666.00000,711.0000


In [68]:
train_labels = train_dataset.pop('MEDV')
test_labels = test_dataset.pop('MEDV')

In [69]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [70]:
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

history = model.fit(normed_train_data, train_labels, epochs=1000,
                    validation_split = 0.2, verbose=1, callbacks=[early_stop])

Epoch 1/1000
11/11 [==============================] - 0s 11ms/step - loss: 374.8689 - mae: 16.9271 - mse: 374.8689 - val_loss: 177.6398 - val_mae: 11.7763 - val_mse: 177.6398
Epoch 2/1000
11/11 [==============================] - 0s 2ms/step - loss: 180.7173 - mae: 10.7068 - mse: 180.7173 - val_loss: 116.9252 - val_mae: 8.9608 - val_mse: 116.9252
Epoch 3/1000
11/11 [==============================] - 0s 3ms/step - loss: 52.2304 - mae: 5.5296 - mse: 52.2304 - val_loss: 56.7769 - val_mae: 6.2873 - val_mse: 56.7769
Epoch 4/1000
11/11 [==============================] - 0s 3ms/step - loss: 49.3486 - mae: 5.1243 - mse: 49.3486 - val_loss: 57.6844 - val_mae: 5.6874 - val_mse: 57.6844
Epoch 5/1000
11/11 [==============================] - 0s 4ms/step - loss: 27.5581 - mae: 3.7497 - mse: 27.5581 - val_loss: 26.0655 - val_mae: 3.2384 - val_mse: 26.0655
Epoch 6/1000
11/11 [==============================] - 0s 3ms/step - loss: 23.3366 - mae: 3.4059 - mse: 23.3366 - val_loss: 20.5034 - val_mae: 3.1358

In [16]:
model.save("files/boston_model.h5")

In [71]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

array([[21.670946],
       [47.278618],
       [24.05969 ],
       [ 8.908265],
       [20.650394],
       [19.37328 ],
       [21.825243],
       [19.240065],
       [16.636889],
       [17.801283]], dtype=float32)